In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sps
import matplotlib.pyplot as pyplot
import time

from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Utils.DataReader import load_urm, load_icm, load_target
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

In [9]:
URM_all = load_urm()

URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.85)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 4300 (10.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 3118 ( 7.5%) Users that have less than 1 test interactions


In [10]:
recommender_class = SLIMElasticNetRecommender

In [11]:
import os

output_folder_path = "Experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

n_cases = 50
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"
cutoff_to_optimize = 10

In [12]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {
    "l1_ratio": Real(low = 0.001, high = 0.01, prior = 'log-uniform'),
    "alpha": Real(low = 0.01, high = 0.1, prior = 'log-uniform'),
    "positive_only": Categorical([True]),
    "topK": Integer(200,450)
}

In [13]:
earlystopping_keywargs = {"validation_every_n": 5,
                          "stop_on_validation": True,
                          "evaluator_object": evaluator_validation,
                          "lower_validations_allowed": 5,
                          "validation_metric": metric_to_optimize,
                          }

In [14]:
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

#create a bayesian optimizer object, we pass the recommender and the evaluator
hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                           evaluator_validation=evaluator_validation)

In [15]:
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

#provide data needed to create instance of model (one on URM_train, the other on URM_all)
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = earlystopping_keywargs
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_all],
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = earlystopping_keywargs
)

In [16]:
hyperparameterSearch.search(recommender_input_args = recommender_input_args,
                            recommender_input_args_last_test = recommender_input_args_last_test,
                            hyperparameter_search_space = hyperparameters_range_dictionary,
                            n_cases = n_cases,
                            n_random_starts = n_random_starts,
                            save_model = "last",
                            output_folder_path = output_folder_path, # Where to save the results
                            output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                            metric_to_optimize = metric_to_optimize,
                            cutoff_to_optimize = cutoff_to_optimize,
                            )

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'l1_ratio': 0.003183567349249954, 'alpha': 0.07166923356367623, 'positive_only': True, 'topK': 245}
SLIMElasticNetRecommender: URM Detected 1 ( 0.0%) users with no interactions.
SearchBayesianSkopt: Config 0 Exception. Config: {'l1_ratio': 0.003183567349249954, 'alpha': 0.07166923356367623, 'positive_only': True, 'topK': 245} - Exception: Traceback (most recent call last):
  File "C:\Users\Luca\DataspellProjects\Recommender-Systems-Challenge-2022\HyperparameterTuning\SearchAbstractClass.py", line 468, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(current_fit_hyperparameters_dict, was_already_evaluated_flag, was_already_evaluated_index)
  File "C:\Users\Luca\DataspellProjects\Recommender-Systems-Challenge-2022\HyperparameterTuning\SearchAbstractClass.py", line 326, in _evaluate_on_validation
    recommender_instance, train_time = self._fit_model

Traceback (most recent call last):
  File "C:\Users\Luca\DataspellProjects\Recommender-Systems-Challenge-2022\HyperparameterTuning\SearchAbstractClass.py", line 468, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(current_fit_hyperparameters_dict, was_already_evaluated_flag, was_already_evaluated_index)
  File "C:\Users\Luca\DataspellProjects\Recommender-Systems-Challenge-2022\HyperparameterTuning\SearchAbstractClass.py", line 326, in _evaluate_on_validation
    recommender_instance, train_time = self._fit_model(current_fit_hyperparameters)
  File "C:\Users\Luca\DataspellProjects\Recommender-Systems-Challenge-2022\HyperparameterTuning\SearchAbstractClass.py", line 297, in _fit_model
    recommender_instance.fit(*self.recommender_input_args.FIT_POSITIONAL_ARGS,
  File "C:\Users\Luca\miniconda3\lib\site-packages\sklearn\utils\_testing.py", line 320, in wrapper
    return fn(*args, **kwargs)
TypeError: fit() got an unexpected keyword argument 'val

Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.5434
Function value obtained: 65504.0000
Current minimum: 65504.0000
Iteration No: 2 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'l1_ratio': 0.00886093806520026, 'alpha': 0.023103099725073482, 'positive_only': True, 'topK': 420}
SLIMElasticNetRecommender: URM Detected 1 ( 0.0%) users with no interactions.
SearchBayesianSkopt: Config 1 Exception. Config: {'l1_ratio': 0.00886093806520026, 'alpha': 0.023103099725073482, 'positive_only': True, 'topK': 420} - Exception: Traceback (most recent call last):
  File "C:\Users\Luca\DataspellProjects\Recommender-Systems-Challenge-2022\HyperparameterTuning\SearchAbstractClass.py", line 468, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(current_fit_hyperparameters_dict, was_already_evaluated_flag, was_already_evaluated_index)
  File "C:\Users\Luca\DataspellProjects\Recommender-Systems-Challenge-202

Traceback (most recent call last):
  File "C:\Users\Luca\DataspellProjects\Recommender-Systems-Challenge-2022\HyperparameterTuning\SearchAbstractClass.py", line 468, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(current_fit_hyperparameters_dict, was_already_evaluated_flag, was_already_evaluated_index)
  File "C:\Users\Luca\DataspellProjects\Recommender-Systems-Challenge-2022\HyperparameterTuning\SearchAbstractClass.py", line 326, in _evaluate_on_validation
    recommender_instance, train_time = self._fit_model(current_fit_hyperparameters)
  File "C:\Users\Luca\DataspellProjects\Recommender-Systems-Challenge-2022\HyperparameterTuning\SearchAbstractClass.py", line 297, in _fit_model
    recommender_instance.fit(*self.recommender_input_args.FIT_POSITIONAL_ARGS,
  File "C:\Users\Luca\miniconda3\lib\site-packages\sklearn\utils\_testing.py", line 320, in wrapper
    return fn(*args, **kwargs)
TypeError: fit() got an unexpected keyword argument 'val

Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.1462
Function value obtained: 65504.0000
Current minimum: 65504.0000
Iteration No: 4 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'l1_ratio': 0.003196171350897768, 'alpha': 0.03915950644390028, 'positive_only': True, 'topK': 237}
SLIMElasticNetRecommender: URM Detected 1 ( 0.0%) users with no interactions.
SearchBayesianSkopt: Config 3 Exception. Config: {'l1_ratio': 0.003196171350897768, 'alpha': 0.03915950644390028, 'positive_only': True, 'topK': 237} - Exception: Traceback (most recent call last):
  File "C:\Users\Luca\DataspellProjects\Recommender-Systems-Challenge-2022\HyperparameterTuning\SearchAbstractClass.py", line 468, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(current_fit_hyperparameters_dict, was_already_evaluated_flag, was_already_evaluated_index)
  File "C:\Users\Luca\DataspellProjects\Recommender-Systems-Challenge-202

...

Iteration No: 11 ended. Evaluation done at random point.
Time taken: 0.1272
Function value obtained: 65504.0000
Current minimum: 65504.0000
Iteration No: 12 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'l1_ratio': 0.0022598055108969638, 'alpha': 0.032409814536710456, 'positive_only': True, 'topK': 290}
SLIMElasticNetRecommender: URM Detected 1 ( 0.0%) users with no interactions.
SearchBayesianSkopt: Config 11 Exception. Config: {'l1_ratio': 0.0022598055108969638, 'alpha': 0.032409814536710456, 'positive_only': True, 'topK': 290} - Exception: Traceback (most recent call last):
  File "C:\Users\Luca\DataspellProjects\Recommender-Systems-Challenge-2022\HyperparameterTuning\SearchAbstractClass.py", line 468, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(current_fit_hyperparameters_dict, was_already_evaluated_flag, was_already_evaluated_index)
  File "C:\Users\Luca\DataspellProjects\Recommender-Systems-Challe

Traceback (most recent call last):
  File "C:\Users\Luca\DataspellProjects\Recommender-Systems-Challenge-2022\HyperparameterTuning\SearchAbstractClass.py", line 468, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(current_fit_hyperparameters_dict, was_already_evaluated_flag, was_already_evaluated_index)
  File "C:\Users\Luca\DataspellProjects\Recommender-Systems-Challenge-2022\HyperparameterTuning\SearchAbstractClass.py", line 326, in _evaluate_on_validation
    recommender_instance, train_time = self._fit_model(current_fit_hyperparameters)
  File "C:\Users\Luca\DataspellProjects\Recommender-Systems-Challenge-2022\HyperparameterTuning\SearchAbstractClass.py", line 297, in _fit_model
    recommender_instance.fit(*self.recommender_input_args.FIT_POSITIONAL_ARGS,
  File "C:\Users\Luca\miniconda3\lib\site-packages\sklearn\utils\_testing.py", line 320, in wrapper
    return fn(*args, **kwargs)
TypeError: fit() got an unexpected keyword argument 'val

Iteration No: 13 ended. Evaluation done at random point.
Time taken: 0.3258
Function value obtained: 65504.0000
Current minimum: 65504.0000
Iteration No: 14 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'l1_ratio': 0.0015202986495068402, 'alpha': 0.02659997255409595, 'positive_only': True, 'topK': 257}
SLIMElasticNetRecommender: URM Detected 1 ( 0.0%) users with no interactions.
SearchBayesianSkopt: Config 13 Exception. Config: {'l1_ratio': 0.0015202986495068402, 'alpha': 0.02659997255409595, 'positive_only': True, 'topK': 257} - Exception: Traceback (most recent call last):
  File "C:\Users\Luca\DataspellProjects\Recommender-Systems-Challenge-2022\HyperparameterTuning\SearchAbstractClass.py", line 468, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(current_fit_hyperparameters_dict, was_already_evaluated_flag, was_already_evaluated_index)
  File "C:\Users\Luca\DataspellProjects\Recommender-Systems-Challeng

Traceback (most recent call last):
  File "C:\Users\Luca\DataspellProjects\Recommender-Systems-Challenge-2022\HyperparameterTuning\SearchAbstractClass.py", line 468, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(current_fit_hyperparameters_dict, was_already_evaluated_flag, was_already_evaluated_index)
  File "C:\Users\Luca\DataspellProjects\Recommender-Systems-Challenge-2022\HyperparameterTuning\SearchAbstractClass.py", line 326, in _evaluate_on_validation
    recommender_instance, train_time = self._fit_model(current_fit_hyperparameters)
  File "C:\Users\Luca\DataspellProjects\Recommender-Systems-Challenge-2022\HyperparameterTuning\SearchAbstractClass.py", line 297, in _fit_model
    recommender_instance.fit(*self.recommender_input_args.FIT_POSITIONAL_ARGS,
  File "C:\Users\Luca\miniconda3\lib\site-packages\sklearn\utils\_testing.py", line 320, in wrapper
    return fn(*args, **kwargs)
TypeError: fit() got an unexpected keyword argument 'val

SearchBayesianSkopt: Search interrupted. No valid config was found during the initial random initialization



In [17]:
from Recommenders.DataIO import DataIO

#explore the results of the search
data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")

search_metadata.keys()

dict_keys(['algorithm_name_recommender', 'algorithm_name_search', 'cutoff_to_optimize', 'exception_list', 'hyperparameters_best', 'hyperparameters_best_index', 'hyperparameters_df', 'metric_to_optimize', 'result_on_earlystopping_df', 'result_on_last', 'result_on_test_best', 'result_on_test_df', 'result_on_validation_best', 'result_on_validation_df', 'time_df', 'time_on_last_df', 'time_on_test_avg', 'time_on_test_total', 'time_on_train_avg', 'time_on_train_total', 'time_on_validation_avg', 'time_on_validation_total'])

In [18]:
hyperparameters_df = search_metadata["hyperparameters_df"]
hyperparameters_df

,l1_ratio,alpha,positive_only,topK
0,0.003184,0.071669,True,245
1,0.008861,0.023103,True,420
2,0.00422,0.072511,True,374
3,0.003196,0.03916,True,237
4,0.001331,0.01794,True,329
5,0.001434,0.021192,True,446
6,0.001355,0.038925,True,397
7,0.003064,0.026527,True,367
8,0.002845,0.02035,True,366
9,0.002891,0.013004,True,339


In [21]:
result_on_validation_df = search_metadata["result_on_validation_df"]
result_on_validation_df

In [22]:
best_hyperparameters = search_metadata["hyperparameters_best"]
best_hyperparameters

In [ ]:
recommender = SLIMElasticNetRecommender(URM_all)
recommender.fit(...)

In [ ]:
recommender.save_model(output_folder_path, file_name = recommender.RECOMMENDER_NAME + "_my_own_save.zip" )

In [ ]:
test_users = pd.read_csv('../Dataset/data_target_users_test.csv')

In [ ]:
user_id = test_users['user_id']
recommendations = []
for user in user_id:
    recommendations.append(recommender.recommend(user, cutoff=10))

In [ ]:
for index in range(len(recommendations)):
    recommendations[index]=np.array(recommendations[index])

test_users['item_list']= recommendations
test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])
test_users.to_csv('..\Submissions\Submission_04_RP3Beta_MixedRatings.csv', index=False)